In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../scripts')))

In [3]:
from utils import *
from plottings import *

In [6]:
df = pd.read_csv('../../data/train.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_16240\1363763938.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/train.csv')


In [7]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Store,int64,0,1017209,1115,"{1: 942, 726: 942, 708: 942, 709: 942, 713: 94..."
1,DayOfWeek,int64,0,1017209,7,"{5: 145845, 4: 145845, 3: 145665, 2: 145664, 1..."
2,Date,object,0,1017209,942,"{'2015-07-31': 1115, '2013-11-06': 1115, '2013..."
3,Sales,int64,0,1017209,21734,"{0: 172871, 5674: 215, 5558: 197, 5483: 196, 6..."
4,Customers,int64,0,1017209,4086,"{0: 172869, 560: 2414, 576: 2363, 603: 2337, 5..."
5,Open,int64,0,1017209,2,"{1: 844392, 0: 172817}"
6,Promo,int64,0,1017209,2,"{0: 629129, 1: 388080}"
7,StateHoliday,object,0,1017209,5,"{'0': 855087, 0: 131072, 'a': 20260, 'b': 6690..."
8,SchoolHoliday,int64,0,1017209,2,"{0: 835488, 1: 181721}"


### NO null Value